In [1]:
import numpy as np
import math

In [2]:
SUN_RANK = [7, 8, 9, 11, 12, 13, 10, 1]
SUN_SCORES = {
    7: 0,
    8: 0,
    9: 0,
    11: 2,
    12: 3,
    13: 4,
    10: 10,
    1: 11
}

In [3]:
np.random.seed(42)

# SUIT ORDER: HEARTS, SPADES, DIAMONDS, CLUBS

player_cards = np.array([rank + suit * 13 for suit in (0, 1, 2, 3) for rank in (1, 7, 8, 9, 10, 11, 12, 13)], dtype='uint8')
np.random.shuffle(player_cards)
player_cards = player_cards.reshape(4, 8)

player_cards

array([[50, 26, 40, 33, 14, 20, 51, 46],
       [23,  1, 10, 27, 11, 24, 22, 39],
       [ 7,  8, 47,  9, 37, 48, 38, 34],
       [52, 36, 13, 21, 25, 49, 35, 12]], dtype=uint8)

In [39]:
def get_suit(card):
    return (card - 1) // 13

def get_rank(card):
    return (card - 1) % 13 + 1

def has_suit(cards, suit):
    for card in cards:
        if get_suit(card) == suit:
            return True
    return False

def get_trick_winner(trick, starter=0):
    trick_suit = get_suit(trick[0])
    
    winner = starter
    max_rank_idx = SUN_RANK.index(get_rank(trick[0]))
    for i, card in enumerate(trick):
        if get_suit(card) == trick_suit and SUN_RANK.index(get_rank(trick[i])) > max_rank_idx:
            max_rank_idx = SUN_RANK.index(get_rank(trick[i]))
            winner = (i + starter) % 4

    return winner

def calculate_score(played_cards):
    cur_starter = 0
    total_score = 0
    
    for i in range(0, 29, 4):
        cur_starter = get_trick_winner(played_cards[i: i + 4], cur_starter)
        round_score = 0
        for j in range(4):
            card_rank = get_rank(played_cards[i + j])
            round_score += SUN_SCORES[card_rank]

        if cur_starter == 0 or cur_starter == 2:
            total_score += round_score

        if i == 28 and (cur_starter == 0 or cur_starter == 2):
            total_score += 10

    return total_score

In [170]:
class State:
    def __init__(self, played_cards, current_cards, current_player, num_of_played_cards, trick_starter, parent):
        self.state_score = 0
        self.state_visits = 0
        self.lowest_score = 130
        self.highest_score = 0
        self.played_cards = played_cards # 1D array of cards played in order
        self.current_cards = current_cards # 2D array of the remaining cards in each player's hand
        self.current_player = current_player
        self.num_of_played_cards = num_of_played_cards
        self.trick_starter = trick_starter
        self.child_states = {
            # action: new_state
        }
        self.parent = parent

    def expand(self):
        if self.num_of_played_cards % 4 == 0:
            # first card in the trick
            for i, card in enumerate(self.current_cards[self.current_player]):
                if card == 0:
                    continue
                    
                updated_current_cards = self.current_cards.copy()
                updated_current_cards[self.current_player][i] = 0

                updated_played_cards = self.played_cards.copy()
                updated_played_cards[self.num_of_played_cards] = card

                self.child_states[card] = State(updated_played_cards, updated_current_cards, (self.current_player + 1) % 4,
                                                self.num_of_played_cards + 1, self.trick_starter, self)

        else:
            trick_first_card = self.played_cards[(self.num_of_played_cards // 4) * 4]
            trick_suit = get_suit(trick_first_card)

            player_has_suit = has_suit(self.current_cards[self.current_player], trick_suit)
            for i, card in enumerate(self.current_cards[self.current_player]):
                if player_has_suit and trick_suit == get_suit(card) or not player_has_suit:
                    if card == 0:
                        continue
                        
                    updated_current_cards = self.current_cards.copy()
                    updated_current_cards[self.current_player][i] = 0
    
                    updated_played_cards = self.played_cards.copy()
                    updated_played_cards[self.num_of_played_cards] = card

                    if self.num_of_played_cards % 4 == 3: # last card in the trick
                        updated_trick_starter = (get_trick_winner(updated_played_cards[self.num_of_played_cards - 3: self.num_of_played_cards + 1]) + self.trick_starter) % 4
                    
                        self.child_states[card] = State(updated_played_cards, updated_current_cards, updated_trick_starter,
                                                    self.num_of_played_cards + 1, updated_trick_starter, self)
                    else:
                        self.child_states[card] = State(updated_played_cards, updated_current_cards, (self.current_player + 1) % 4,
                                                self.num_of_played_cards + 1, self.trick_starter, self)

    def ucb1_score(self, child_state, maximizing):
        if child_state.state_visits == 0:
            return float('inf')
        else:
            exploitation_term = child_state.state_score / child_state.state_visits * (1 if maximizing else -1)
            constant = 130 * 2 ** 0.5
            prior_constant = 10000000
            exploration_term = math.sqrt(math.log(self.state_visits) / child_state.state_visits)
            
            return exploitation_term + constant * exploration_term + prior_constant * ((self.highest_score * self.lowest_score) if maximizing else ((130 - self.lowest_score) * (130 - self.highest_score)))
    
    def get_optimal_node(self):
        if self.current_player == 1 or self.current_player == 3:
            maximizing = False
        else:
            maximizing = True
        
        optimal_action = None
        optimal_child_state = None
        optimal_ucb1_score = -float('inf')
            
        for action, child_state in self.child_states.items():
            current_child_ucb1_score = self.ucb1_score(child_state, maximizing)
            if current_child_ucb1_score > optimal_ucb1_score:
                optimal_ucb1_score = current_child_ucb1_score
                optimal_action = action
                optimal_child_state = child_state

        return optimal_child_state

    def pick_random_card(self, current_cards, current_player, suit=None):
        current_player_cards = current_cards[current_player]
        if suit is None:
            card = np.random.choice(current_player_cards)
            while card == 0:
                card = np.random.choice(current_player_cards)
            idx = np.where(current_player_cards == card)[0][0]
            return card, idx
        else:
            card = np.random.choice(current_player_cards)
            while card == 0 or get_suit(card) != suit:
                card = np.random.choice(current_player_cards)
            idx = np.where(current_player_cards == card)[0][0]
            return card, idx
    
    def rollout(self):
        updated_played_cards = self.played_cards.copy()
        updated_current_cards = self.current_cards.copy()
        updated_current_player = self.current_player
        updated_trick_starter = self.trick_starter
        
        for i in range(self.num_of_played_cards, 32):
            if i % 4 == 0:
                card, idx = self.pick_random_card(updated_current_cards, updated_current_player)
            else:
                trick_first_card = updated_played_cards[(i // 4) * 4]
                trick_suit = get_suit(trick_first_card)

                player_has_suit = has_suit(updated_current_cards[updated_current_player], trick_suit)
                
                if player_has_suit:
                    card, idx = self.pick_random_card(updated_current_cards, updated_current_player, trick_suit)
                else:
                    card, idx = self.pick_random_card(updated_current_cards, updated_current_player)
                    
            
            updated_played_cards[i] = card
            updated_current_cards[updated_current_player][idx] = 0

            if i % 4 == 3:
                updated_trick_starter = (get_trick_winner(updated_played_cards[i - 3: i + 1]) + updated_trick_starter) % 4
                updated_current_player = updated_trick_starter
            else:
                updated_current_player = (updated_current_player + 1) % 4

        rollout_score = calculate_score(updated_played_cards)

        self.lowest_score = min(self.lowest_score, rollout_score)
        self.highest_score = max(self.highest_score, rollout_score)

        self.state_score += rollout_score
        self.state_visits += 1
        
        cur_parent = self.parent

        while cur_parent != None:
            cur_parent.state_score += rollout_score
            cur_parent.state_visits += 1
            
            cur_parent.lowest_score = min(cur_parent.lowest_score, rollout_score)
            cur_parent.highest_score = max(cur_parent.highest_score, rollout_score)
            cur_parent = cur_parent.parent

In [172]:
initial_state = State(played_cards=np.zeros(32, dtype='uint8'), current_cards=player_cards.copy(), current_player=0, num_of_played_cards=0,
                     trick_starter=0, parent=None)

initial_state.expand()

ITERATIONS = 10000

root_state = initial_state

for card in range(32):
    for i in range(ITERATIONS): 
        current_state = root_state
        while current_state.child_states:
            current_state = current_state.get_optimal_node()
        if current_state.state_visits == 0:
            current_state.rollout()
        else:
            if current_state.num_of_played_cards == 32:
                current_state.rollout()
            else:
                current_state.expand()
                current_state = current_state.get_optimal_node()
                current_state.rollout()
    
    # SUIT ORDER: 0. HEARTS, 1. SPADES, 2. DIAMONDS, 3. CLUBS
    max_visits = 0
    card = 0
    for k, v in root_state.child_states.items():
        if max_visits < v.state_visits:
            max_visits = v.state_visits
            card = k
        print(f"Card {k}, which is the {get_rank(k)} of {get_suit(k)}, yielded a total of {v.state_score} from {v.state_visits} visits for an average of {v.state_score / v.state_visits}, a min of {v.lowest_score}, and a max of {v.highest_score}")
    root_state = root_state.child_states[card]
    print(root_state.played_cards) 

Card 50, which is the 11 of 3, yielded a total of 19812 from 614 visits for an average of 32.267100977198695, a min of 0, and a max of 118
Card 26, which is the 13 of 1, yielded a total of 14745 from 496 visits for an average of 29.72782258064516, a min of 0, and a max of 121
Card 40, which is the 1 of 3, yielded a total of 271072 from 5718 visits for an average of 47.40678558936691, a min of 15, and a max of 121
Card 33, which is the 7 of 2, yielded a total of 17869 from 570 visits for an average of 31.349122807017544, a min of 0, and a max of 114
Card 14, which is the 1 of 1, yielded a total of 25186 from 736 visits for an average of 34.22010869565217, a min of 11, and a max of 118
Card 20, which is the 7 of 1, yielded a total of 17215 from 554 visits for an average of 31.074007220216608, a min of 0, and a max of 117
Card 51, which is the 12 of 3, yielded a total of 20322 from 626 visits for an average of 32.46325878594249, a min of 0, and a max of 121
Card 46, which is the 7 of 3, y

In [ ]:
current_state = initial_state

for i in range(32):
    max_action = 0
    max_visits = 0
    for action, child in current_state.child_states.items():
        if child.state_visits > max_visits:
            max_action = action
            max_visits = child.state_visits
    if current_state.num_of_played_cards != 32:
        print(current_state.num_of_played_cards)
        print(current_state.played_cards)
        current_state = current_state.child_states[max_action]

print(current_state.played_cards)

In [63]:
SUITS = {
    0: "Hearts",
    1: "Spades",
    2: "Diamonds",
    3: "Clubs",
}

for i, card in enumerate(root_state.played_cards):
    print(f"Card #{i + 1}: {get_rank(card)} of {SUITS[get_suit(card)]}")

Card #1: 1 of Clubs
Card #2: 9 of Spades
Card #3: 8 of Clubs
Card #4: 13 of Clubs
Card #5: 7 of Clubs
Card #6: 11 of Spades
Card #7: 9 of Clubs
Card #8: 10 of Clubs
Card #9: 10 of Diamonds
Card #10: 7 of Diamonds
Card #11: 1 of Diamonds
Card #12: 8 of Diamonds
Card #13: 11 of Hearts
Card #14: 7 of Hearts
Card #15: 12 of Hearts
Card #16: 13 of Spades
Card #17: 13 of Hearts
Card #18: 7 of Spades
Card #19: 10 of Hearts
Card #20: 8 of Hearts
Card #21: 1 of Hearts
Card #22: 9 of Hearts
Card #23: 12 of Spades
Card #24: 12 of Clubs
Card #25: 13 of Diamonds
Card #26: 11 of Diamonds
Card #27: 9 of Diamonds
Card #28: 11 of Clubs
Card #29: 10 of Spades
Card #30: 12 of Diamonds
Card #31: 8 of Spades
Card #32: 1 of Spades


In [176]:
main = initial_state.child_states[40].child_states[22].child_states[47].child_states[52].child_states[51]#.child_states[23].child_states[47].child_states[52].child_states[26]
for k, v in main.child_states.items():
    print(f"Card {k}, which is the {get_rank(k)} of {get_suit(k)}, yielded a total of {v.state_score} from {v.state_visits} visits for an average of {v.state_score / v.state_visits}, a min of {v.lowest_score}, and a max of {v.highest_score}")

    print(main.get_optimal_node().ucb1_score(v, True))

Card 23, which is the 10 of 1, yielded a total of 62209 from 1350 visits for an average of 46.080740740740744, a min of 15, and a max of 107
15900000059.41277
Card 1, which is the 1 of 0, yielded a total of 85209 from 1963 visits for an average of 43.407539480387165, a min of 15, and a max of 106
15900000054.463665
Card 10, which is the 10 of 0, yielded a total of 80107 from 1825 visits for an average of 43.894246575342464, a min of 15, and a max of 107
15900000055.360767
Card 27, which is the 1 of 2, yielded a total of 56850 from 1211 visits for an average of 46.94467382328654, a min of 15, and a max of 106
15900000061.021055
Card 11, which is the 11 of 0, yielded a total of 57568 from 1229 visits for an average of 46.841334418226204, a min of 15, and a max of 115
15900000060.814255
Card 24, which is the 11 of 1, yielded a total of 11660267 from 268052 visits for an average of 43.50001865309716, a min of 15, and a max of 115
15900000044.446156
Card 39, which is the 13 of 2, yielded a 

In [ ]:

for i, card in enumerate(player_cards.reshape(32)):
    print(f"Card #{i + 1}: {get_rank(card)} of {SUITS[get_suit(card)]}")

In [132]:
calculate_score(root_state.played_cards)

41

In [174]:
root_state.played_cards

array([40, 22, 47, 52, 51, 24, 48, 49, 36, 33, 27, 34,  1,  8, 12, 20, 10,
        9, 13, 26, 11,  7, 25, 50, 39, 37, 35, 46, 23, 38, 21, 14],
      dtype=uint8)

In [ ]:
get_trick_winner([39, 37, 35, 33], 1)